### STEP: 1

Import necessary libraries for **Predicting Future Stock Prices**

In [64]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import requests

import warnings
warnings.filterwarnings('ignore')

# Libraries to Extract Stock Price Historical Data 
# from Yahoo! Finance using Tickers from Wikipedia
import yfinance as yf
from pandas.tseries.offsets import DateOffset
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar

# Library for Technical Analysis
import pandas_ta as ta

# Libraries for ML Analysis using Linear Models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge

scaler = StandardScaler()
today = datetime.today().strftime('%Y-%m-%d')

### STEP: 2

Defining various functions used in the code for 
* extracting data, 
* adding technical indicators as features,
* predicting **Close** price and **VWAP** volume-weighted average price,
* using linear models for predicting with best model output.

Best model is selected based on minimum MAPE & RMSE

In [65]:
# Extracting Stock Price Historical Data from Yahoo! Finance using Tickers from Wikipedia
        
def extract_stock_data(stock_name, start_date):
    row_index = ticker_df.query(f"Security == '{stock_name}'").index[0]
    stock_ticker = ticker_list[row_index]
    OHLCV_df = yf.download(stock_ticker,start_date,today)
    OHLCV_df.index = pd.to_datetime(OHLCV_df.index)
    print(f"{stock_ticker}:{stock_name}")
    #display(OHLCV_df.head(3))
    #display(OHLCV_df.tail(3))
    return(OHLCV_df)

# Feature Engineering _ additionals features for ML Analysis

def technical_indicators():
    OHLCV_df = extract_stock_data(stock_name, start_date)
    # VWAP modified to use Annual cummulative sum
    OHLCV_df['typical_price'] = (OHLCV_df['High']+OHLCV_df['Low']+OHLCV_df['Close'])/3
    OHLCV_df['vw_typical_price'] = OHLCV_df['typical_price']*OHLCV_df['Volume']
    OHLCV_df['vwap_annual'] = OHLCV_df.groupby(OHLCV_df.index.year)['vw_typical_price'].cumsum() / OHLCV_df.groupby(OHLCV_df.index.year)['Volume'].cumsum()
    # Daily Returns
    OHLCV_df['daily_returns'] = OHLCV_df['Close'].pct_change()
    # 21Day Volatility
    OHLCV_df['volatility_21'] = OHLCV_df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    OHLCV_df['sma_21'] = OHLCV_df.ta.sma(length=21)
    OHLCV_df['sma_50'] = OHLCV_df.ta.sma(length=50)
    # Exponential Moving Avg 21Days & 50Days
    OHLCV_df['ema_21'] = OHLCV_df.ta.ema(length=21)
    OHLCV_df['ema_50'] = OHLCV_df.ta.ema(length=50)
    # RSI Momemtum Indicator 14Days
    OHLCV_df['rsi_14'] = OHLCV_df.ta.rsi()
    # MACD Momentum Indicator
    OHLCV_df[['ema_12', 'ema_26', 'MACD_signal']] = OHLCV_df.ta.macd()
    # Bolinger Bands %B Indicator
    OHLCV_df[['BB_lower','BB_middle','BB_upper','BBP','BB%B']]= OHLCV_df.ta.bbands()
    # On-Balance Volume in Millions
    OHLCV_df['OBV_in_million'] = OHLCV_df.ta.obv()/1000000
    # Average Directional Index 14Days
    OHLCV_df[['adx_14','dmp_14','dmn_14']] = OHLCV_df.ta.adx()
    # Dropping trivial Columns
    OHLCV_append_df = OHLCV_df.drop(columns = ['Open','High','Low','Adj Close',
                                               'typical_price','vw_typical_price',
                                               'BBP','dmp_14','dmn_14']).dropna()
    #display(OHLCV_append_df.tail())
    
    return(OHLCV_append_df)


In [69]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    list_of_models = [Ridge(),Lasso(),MultiTaskElasticNet(),
                  LassoLars(),BayesianRidge()]
    target_predictions_close_df = target_test_close.copy()
    for model in list_of_models:
        model_instance_close = model
        model_instance_close.fit(predictors_train_scaled,target_train_close)
        model_prediction_close = model_instance_close.predict(predictors_test_scaled)
        target_predictions_close_df[f'{model}_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    for model in list_of_models:
        model_instance_vwap = model
        model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
        model_prediction_vwap = model_instance_vwap.predict(predictors_test_scaled)
        target_predictions_vwap_df[f'{model}_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    display(target_predictions_df.tail())
    display(target_predictions_close_df.hvplot())
    display(target_predictions_vwap_df.hvplot())
    return(target_predictions_close_df,target_predictions_vwap_df,target_predictions_df)



In [70]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def linear_model_predictions():
    display(extract_stock_data(stock_name, start_date).head(3))
    display(extract_stock_data(stock_name, start_date).tail(3))
    display(technical_indicators().tail())
    today = datetime.today().strftime('%Y-%m-%d')
    target_close_predictions_df,target_vwap_predictions_df,target_predictions_df = predict_close_vwap()
    column_range_close = list(target_close_predictions_df.columns)
    column_range_vwap = list(target_vwap_predictions_df.columns)
    dict_close = {}
    dict_vwap = {}
    for i in column_range_close[1:]:
        rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])**2))
        mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'Close']))*100
        dict_close[f'{i}'] = [rmse_close,mape_close]
    for i in column_range_vwap[1:]:  
        rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])**2))
        mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'vwap_annual']))*100
        dict_vwap[f'{i}'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])
    error_measures_close = error_measures_close.sort_values(by=['MAPE_close'])
    error_measures_vwap = error_measures_vwap.sort_values(by=['MAPE_vwap'])

    display(error_measures_close)
    display(error_measures_close.hvplot.bar(rot=90))
    display(error_measures_vwap)
    display(error_measures_vwap.hvplot.bar(rot=90))

    best_estimate_close = error_measures_close.index[0]
    best_estimate_vwap = error_measures_vwap.index[0]

    predictions_output = target_predictions_df[['Close','vwap_annual',best_estimate_close, best_estimate_vwap]]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output[best_estimate_close])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output[best_estimate_vwap])

    predictions = predictions_output[['Close','vwap_annual']]
    
    return (predictions)


In [71]:
# Scrapping for List of stocks in S&P500 for Linear Model ML Prediction Analysis
url = input('Enter Wikipedia url for list of S&P500 Companies_')
  
wiki_data = pd.read_html(url)
print(wiki_data)

ticker_df = wiki_data[0]
display(ticker_df.head(2))
display(ticker_df.tail(2))

# Pick Stock from the list for analysis

ticker_list= ticker_df['Symbol'].to_list()
stock_list = ticker_df['Security'].to_list()
print(stock_list)
stock_name = input('Enter Stock_Name from the above displayed list for Analysis:')
start_date = input('Enter Start Date for Historical Data in yyyy-mm-dd:')

stock_predictions = linear_model_predictions()

stock_predictions

Enter Wikipedia url for list of S&P500 Companies_ https://en.wikipedia.org/wiki/List_of_S%26P_500_companies


[    Symbol            Security             GICS Sector  \
0      MMM                  3M             Industrials   
1      AOS         A. O. Smith             Industrials   
2      ABT              Abbott             Health Care   
3     ABBV              AbbVie             Health Care   
4      ACN           Accenture  Information Technology   
..     ...                 ...                     ...   
498    XYL          Xylem Inc.             Industrials   
499    YUM         Yum! Brands  Consumer Discretionary   
500   ZBRA  Zebra Technologies  Information Technology   
501    ZBH       Zimmer Biomet             Health Care   
502    ZTS              Zoetis             Health Care   

                                GICS Sub-Industry    Headquarters Location  \
0                        Industrial Conglomerates    Saint Paul, Minnesota   
1                               Building Products     Milwaukee, Wisconsin   
2                           Health Care Equipment  North Chicago, Il

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
502,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


['3M', 'A. O. Smith', 'Abbott', 'AbbVie', 'Accenture', 'Adobe Inc.', 'Advanced Micro Devices', 'AES Corporation', 'Aflac', 'Agilent Technologies', 'Air Products and Chemicals', 'Airbnb', 'Akamai', 'Albemarle Corporation', 'Alexandria Real Estate Equities', 'Align Technology', 'Allegion', 'Alliant Energy', 'Allstate', 'Alphabet Inc. (Class A)', 'Alphabet Inc. (Class C)', 'Altria', 'Amazon', 'Amcor', 'Ameren', 'American Airlines Group', 'American Electric Power', 'American Express', 'American International Group', 'American Tower', 'American Water Works', 'Ameriprise Financial', 'Ametek', 'Amgen', 'Amphenol', 'Analog Devices', 'Ansys', 'Aon', 'APA Corporation', 'Apple Inc.', 'Applied Materials', 'Aptiv', 'Arch Capital Group', 'Archer-Daniels-Midland', 'Arista Networks', 'Arthur J. Gallagher & Co.', 'Assurant', 'AT&T', 'Atmos Energy', 'Autodesk', 'Automated Data Processing', 'AutoZone', 'AvalonBay Communities', 'Avery Dennison', 'Axon Enterprise', 'Baker Hughes', 'Ball Corporation', 'Bank

...

,RMSE_close,MAPE_close
BayesianRidge()_Close,1.523890,1.049207
Ridge()_Close,1.584738,1.104549
Lasso()_Close,2.049869,1.610111
LassoLars()_Close,2.056684,1.615988
MultiTaskElasticNet()_Close,3.575941,2.608097


:Bars   [index,Variable]   (value)

,RMSE_vwap,MAPE_vwap
BayesianRidge()_vwap_annual,0.314374,0.092573
Ridge()_vwap_annual,0.373182,0.190054
Lasso()_vwap_annual,1.545519,1.257653
LassoLars()_vwap_annual,1.561679,1.268721
MultiTaskElasticNet()_vwap_annual,6.319515,4.624466


:Bars   [index,Variable]   (value)

,Close,vwap_annual
Date,,
2014-06-17,NaN,NaN
2014-06-18,57.282532,53.426578
2014-06-19,57.778576,53.454421
2014-06-20,57.872036,53.483575
2014-06-23,57.951115,53.505335
...,...,...
2024-03-19,137.289993,133.409752
2024-03-20,136.509995,133.459429
2024-03-21,136.570007,133.498327


## Predicting Stock Price for future dates

In [72]:
# Feature Engineering _ for future prediction using stock_predictions as input
# OBV & ADX dropped to limit predicted features

def feature_engineering(df):
    # Daily Returns
    df['daily_returns'] = df['Close'].pct_change()
    # 21Day Volatility
    df['volatility_21'] = df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    df['sma_21'] = df.ta.sma(length=21)
    df['sma_50'] = df.ta.sma(length=50)
    # Exponential Moving Avg 21Days & 50Days
    df['ema_21'] = df.ta.ema(length=21)
    df['ema_50'] = df.ta.ema(length=50)
     # RSI Momemtum Indicator 14Days
    df['rsi_14'] = df.ta.rsi()
    # MACD Momentum Indicator
    df[['ema_12', 'ema_26', 'MACD_signal']] = df.ta.macd()
    # Bolinger Bands %B Indicator
    df[['BB_lower','BB_middle','BB_upper','BBP','BB%B']]= df.ta.bbands()
    # Dropping trivial Columns
    append_df = df.drop(columns = ['BBP']).dropna()
    #trim_df = df[['Close','vwap_annual']]
    return(append_df)


In [73]:
# Creating instance of best fit model _ BayesianRidge()
# By fitting the historical data

def best_fit_model_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns=['Volume','OBV_in_million','adx_14'])
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model = BayesianRidge()
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model}_Close'] = model_prediction_close[-1]

    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model}_vwap_annual'] = model_prediction_vwap[-1]

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    display(target_future_df.tail(1))

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model}_vwap_annual'])

    predictions = target_future_df[['Close','vwap_annual']]
    
    return (predictions)


In [74]:
#Predicting Day1
future_stock_price = best_fit_model_future_prediction(stock_predictions)
# Predicting Stocks on Future Dates _ Next 51 Days
for i in list(range(1,52)):
    future_stock_price = best_fit_model_future_prediction(future_stock_price)

future_stock_price

[*********************100%%**********************]  1 of 1 completed

YUM:Yum! Brands


,Close,BayesianRidge()_Close,vwap_annual,BayesianRidge()_vwap_annual
Date,,,,
2024-03-25,NaN,136.05113,NaN,133.494033


[*********************100%%**********************]  1 of 1 completed

YUM:Yum! Brands


...

YUM:Yum! Brands


,Close,BayesianRidge()_Close,vwap_annual,BayesianRidge()_vwap_annual
Date,,,,
2024-04-12,NaN,135.929676,NaN,133.621498


[*********************100%%**********************]  1 of 1 completed


YUM:Yum! Brands


ValueError: Cannot set a DataFrame with multiple columns to the single column sma_50

In [75]:
display(future_stock_price)

future_stock_price['Close'].hvplot()

,Close,vwap_annual,daily_returns,volatility_21,sma_21
Date,,,,,
2024-03-13,NaN,NaN,NaN,NaN,NaN
2024-03-14,137.160004,133.105930,NaN,NaN,NaN
2024-03-15,136.839996,133.257582,-0.002333,NaN,NaN
2024-03-18,136.270004,133.334620,-0.004165,NaN,NaN
2024-03-19,137.289993,133.409752,0.007485,NaN,NaN
2024-03-20,136.509995,133.459429,-0.005681,NaN,NaN
2024-03-21,136.570007,133.498327,0.000440,NaN,NaN
2024-03-22,136.229996,133.529837,-0.002490,NaN,NaN
2024-03-25,136.071497,133.506420,-0.001163,NaN,NaN


:Curve   [Date]   (Close)

## Predicting Using Regressor Models

In [80]:
# Single Target Regressor Models
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

svr_rbf = SVR(kernel='rbf',C=100,gamma='auto')
#svr_poly = SVR(kernel='poly',C=100,degree=2, gamma='auto')

knn_uniform = KNeighborsRegressor(n_neighbors=2,weights='uniform')
knn_distance = KNeighborsRegressor(n_neighbors=5,weights='distance')

dtree = DecisionTreeRegressor(max_depth=10)

In [81]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_regressor_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    list_of_regressors = [svr_rbf,knn_uniform,knn_distance,dtree]
    target_predictions_close_df = target_test_close.copy()
    for model in list_of_regressors:
        model_instance_close = model
        model_instance_close.fit(predictors_train_scaled,target_train_close)
        model_prediction_close = model_instance_close.predict(predictors_test_scaled)
        target_predictions_close_df[f'{model}_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    for model in list_of_regressors:
        model_instance_vwap = model
        model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
        model_prediction_vwap = model_instance_vwap.predict(predictors_test_scaled)
        target_predictions_vwap_df[f'{model}_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    display(target_predictions_df.tail())
    display(target_predictions_close_df.hvplot())
    display(target_predictions_vwap_df.hvplot())
    return(target_predictions_close_df,target_predictions_vwap_df,target_predictions_df)


In [82]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def regressor_model_predictions():
    display(extract_stock_data(stock_name, start_date).head(3))
    display(extract_stock_data(stock_name, start_date).tail(3))
    display(technical_indicators().tail())
    today = datetime.today().strftime('%Y-%m-%d')
    target_close_predictions_df,target_vwap_predictions_df,target_predictions_df = predict_regressor_close_vwap()
    column_range_close = list(target_close_predictions_df.columns)
    column_range_vwap = list(target_vwap_predictions_df.columns)
    dict_close = {}
    dict_vwap = {}
    for i in column_range_close[1:]:
        rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])**2))
        mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'Close']))*100
        dict_close[f'{i}'] = [rmse_close,mape_close]
    for i in column_range_vwap[1:]:  
        rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])**2))
        mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'vwap_annual']))*100
        dict_vwap[f'{i}'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])
    error_measures_close = error_measures_close.sort_values(by=['MAPE_close'])
    error_measures_vwap = error_measures_vwap.sort_values(by=['MAPE_vwap'])

    display(error_measures_close)
    display(error_measures_close.hvplot.bar(rot=90))
    display(error_measures_vwap)
    display(error_measures_vwap.hvplot.bar(rot=90))

    best_estimate_close = error_measures_close.index[0]
    best_estimate_vwap = error_measures_vwap.index[0]

    predictions_output = target_predictions_df[['Close','vwap_annual',best_estimate_close, best_estimate_vwap]]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output[best_estimate_close])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output[best_estimate_vwap])

    predictions = predictions_output[['Close','vwap_annual']]
    
    return (predictions)



In [83]:
stock_predictions_regr = regressor_model_predictions()

stock_predictions_regr

[*********************100%%**********************]  1 of 1 completed

YUM:Yum! Brands


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,42.854061,43.062546,42.048885,42.106400,33.329346,4735660
2012-01-04,42.034508,42.774982,42.034508,42.393963,33.556969,5013164
2012-01-05,42.329262,42.774982,41.984184,42.717468,33.813061,5059623


[*********************100%%**********************]  1 of 1 completed

YUM:Yum! Brands


...

,RMSE_close,MAPE_close
DecisionTreeRegressor(max_depth=10)_Close,2.523100,1.827676
"SVR(C=100, gamma='auto')_Close",4.509816,3.050024
KNeighborsRegressor(weights='distance')_Close,9.048675,6.294215
KNeighborsRegressor(n_neighbors=2)_Close,9.543567,6.596114


:Bars   [index,Variable]   (value)

,RMSE_vwap,MAPE_vwap
"SVR(C=100, gamma='auto')_vwap_annual",3.018110,1.950669
DecisionTreeRegressor(max_depth=10)_vwap_annual,4.292829,2.482362
KNeighborsRegressor(weights='distance')_vwap_annual,9.395036,7.599995
KNeighborsRegressor(n_neighbors=2)_vwap_annual,9.757014,7.718965


:Bars   [index,Variable]   (value)

,Close,vwap_annual
Date,,
2014-06-17,NaN,NaN
2014-06-18,57.282532,53.426578
2014-06-19,57.778576,53.454421
2014-06-20,57.872036,53.483575
2014-06-23,57.951115,53.505335
...,...,...
2024-03-19,137.289993,133.409752
2024-03-20,136.509995,133.459429
2024-03-21,136.570007,133.498327


In [89]:
# Creating instance of best fit regressor model
# By fitting the historical data

def best_fit_regr_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns=['Volume','OBV_in_million','adx_14'])
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model_close = dtree
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model_close
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model_close}_Close'] = model_prediction_close[-1]

    best_model_vwap = svr_rbf
    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model_vwap
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model_vwap}_vwap_annual'] = model_prediction_vwap[-1]

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    display(target_future_df.tail(1))

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model_close}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model_vwap}_vwap_annual'])

    predictions = target_future_df[['Close','vwap_annual']]
    
    return (predictions)


In [93]:
#Predicting Day1
future_stock_price_regr = best_fit_regr_future_prediction(stock_predictions_regr)
# Predicting Stocks on Future Dates _ Next 51 Days
for i in list(range(1,52)):
    future_stock_price_regr = best_fit_regr_future_prediction(future_stock_price_regr)


[*********************100%%**********************]  1 of 1 completed


YUM:Yum! Brands


,Close,DecisionTreeRegressor(max_depth=10)_Close,vwap_annual,"SVR(C=100, gamma='auto')_vwap_annual"
Date,,,,
2024-03-25,NaN,130.660004,NaN,124.987344


[*********************100%%**********************]  1 of 1 completed


YUM:Yum! Brands


...

YUM:Yum! Brands


,Close,DecisionTreeRegressor(max_depth=10)_Close,vwap_annual,"SVR(C=100, gamma='auto')_vwap_annual"
Date,,,,
2024-04-12,NaN,124.178,NaN,122.74272


[*********************100%%**********************]  1 of 1 completed

YUM:Yum! Brands


ValueError: Cannot set a DataFrame with multiple columns to the single column sma_50

In [94]:
display(future_stock_price_regr)

future_stock_price_regr['Close'].hvplot()

,Close,vwap_annual,daily_returns,volatility_21,sma_21
Date,,,,,
2024-03-13,NaN,NaN,NaN,NaN,NaN
2024-03-14,137.160004,133.105930,NaN,NaN,NaN
2024-03-15,136.839996,133.257582,-0.002333,NaN,NaN
2024-03-18,136.270004,133.334620,-0.004165,NaN,NaN
2024-03-19,137.289993,133.409752,0.007485,NaN,NaN
2024-03-20,136.509995,133.459429,-0.005681,NaN,NaN
2024-03-21,136.570007,133.498327,0.000440,NaN,NaN
2024-03-22,136.229996,133.529837,-0.002490,NaN,NaN
2024-03-25,133.460007,127.707626,-0.020333,NaN,NaN


:Curve   [Date]   (Close)

In [5]:
# Multi-Layer Perceptron Regressor
# with default 'relu' activation and 'adam' solver

from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(random_state=1, max_iter=500)